In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [3]:
uphill_info = pd.read_excel('data/uphill_information_edited.xlsx', sheet_name = '최종편집')

# 혼잡빈도 파일에서 VDS 필터링

In [36]:
VDS = list(uphill_info['VDS_식별번호'].unique())
print(VDS[:3])

['0500VDE11700', '0500VDE11800', '0500VDS11800']


In [37]:
# 파일 이름 가져오기
folder = 'congestion/'
file_list = os.listdir(folder)
print(file_list[:3])

['혼잡빈도_202012.csv', '혼잡빈도_202101.csv', '혼잡빈도_202102.csv']


In [38]:
save_csv_dir = 'congestion_filtered'
os.makedirs(save_csv_dir, exist_ok = True)

In [39]:
total_df = pd.DataFrame()

for file in tqdm(file_list):
    file_path = os.path.join(folder, file)
    congestion = pd.read_csv(file_path, encoding = 'cp949')
    
    VDS_condition = congestion['VDS_ID'].isin(VDS)
    congestion = congestion.loc[VDS_condition].iloc[:, :-1]

    save_csv_path = os.path.join(save_csv_dir, file)
    congestion.to_csv(save_csv_path, encoding = 'cp949')
    
    total_df = pd.concat([total_df, congestion])

total_df = total_df.iloc[:, 1:]
total_csv_path = 'congestion_total.csv'
total_df.to_csv(total_csv_path, encoding = 'cp949')

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:36<00:00,  8.00s/it]


# UPHILL과 CONGESTION을 MERGE하기

In [40]:
total_df = pd.read_csv(total_csv_path, encoding = 'cp949')

In [41]:
total_df = pd.merge(total_df, uphill_info, how = 'left',
                    left_on = ['VDS_ID', '기점종점방향구분코드'], right_on = ['VDS_식별번호', 'direction'])

In [42]:
total_df = total_df.drop(['direction', '구간', 'VDS_식별번호'], axis = 'columns')
total_df.to_csv('uphill_congestion_merged.csv', encoding = 'cp949')

### 21.12.15 부분추가

In [4]:
congestion = pd.read_csv('data/congestion_filtered/혼잡빈도_202111.csv', encoding = 'cp949')

In [5]:
congestion = pd.merge(congestion, uphill_info, how = 'left',
                    left_on = ['VDS_ID', '기점종점방향구분코드'], right_on = ['VDS_식별번호', 'direction'])

In [8]:
congestion = congestion.drop(['direction', '구간', 'VDS_식별번호'], axis = 'columns')
congestion.to_csv('data/202111_uphill_congestion_merged.csv', encoding = 'cp949')